# Testing in Julia
### Kirsten Landsiedel, Stats 244
#### Feb 6, 2025

I am following many examples and suggestions from [Julia manual's testing section](https://docs.julialang.org/en/v1/stdlib/Test/#:~:text=Testing%20Base%20Julia,-Julia%20is%20under&text=If%20you%20build%20Julia%20from,runtests()%20.&text=Run%20the%20Julia%20unit%20tests,of%20strings%2C%20using%20ncores%20processors). 

Testing is crucial in software development because it helps ensure that code behaves as expected, catches errors early, and prevents errors when making changes. This is especially important in large projects. Without testing, even small modifications can introduce unintended side effects that are hard to track down.

## The "Test" Module in Julia

#### The main testing tools in Julia live in the "Test" module. The building blocks of testing in Julia are the @test and @test_throws macros. Call @test with some STATEMENT as below. 

###  Testing Functions: Overall Goals

Eventually, we want to be able to write tests for functions and for packages (which often contain MANY functions). Here is a quick example of wehre we are going (before we dive into the syntax adn mechanics behind testing in Julia).

Suppose you write a function to compute the avergae of a vector of numbers (but it's 3am, yikes) and you are not at your best:

In [1]:
using Test

function average_bad(nums::AbstractVector{<:Real})
    return sum(nums) / (length(nums) - 1)  # Oops! Wrong denominator
end

# Write tests for multiple input/output pairs
@testset "Testing average_bad" begin 
    @test average_bad([0,1]) ≈ 0.5  
    @test average_bad([-1,2]) ≈ 0.5
    @test average_bad([1,2,3]) ≈ 2
end

Testing average_bad: Test Failed at In[1]:9
  Expression: average_bad([0, 1]) ≈ 0.5
   Evaluated: 1.0 ≈ 0.5

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:679 [inlined]
 [2] macro expansion
   @ In[1]:9 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1704 [inlined]
 [4] top-level scope
   @ In[1]:9
Testing average_bad: Test Failed at In[1]:10
  Expression: average_bad([-1, 2]) ≈ 0.5
   Evaluated: 1.0 ≈ 0.5

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:679 [inlined]
 [2] macro expansion
   @ In[1]:10 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1704 [inlined]
 [4] top-level scope
   @ In[1]:9
Testing average_bad: Test Failed at In[1]:11
  Ex

LoadError: [91mSome tests did not pass: 0 passed, 3 failed, 0 errored, 0 broken.[39m

###  General testing syntax in Julia

If the STATEMENT is TRUE, you get a passing result:

In [2]:
@test 1 + 1 == 2

Test Passed

If the STATEMENT is FALSE you get a failure, but Julia also tries to break down where things went wrong. See 'expression' v 'evaluated'.

In [3]:
@test 1 + 1 == 3

Test Failed at In[3]:1
  Expression: 1 + 1 == 3
   Evaluated: 2 == 3



LoadError: [91mThere was an error during testing[39m

If the STATEMENT cannot be evaluated, Julia will throw an error and stop execution of code.

In [4]:
@test 1 + fish == 3 

Error During Test at In[4]:1
  Test threw exception
  Expression: 1 + fish == 3
  UndefVarError: `fish` not defined in `Main`
  Suggestion: check for spelling errors or missing imports.
  Stacktrace:
   [1] macro expansion
     @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:676 [inlined]
   [2] top-level scope
     @ In[4]:1


LoadError: [91mThere was an error during testing[39m

### Approximate Testing:

Remember to use approximate testing when appropriate to avoid issues with floating point precision.

In [5]:
@test 0.1 + 0.2 == 0.3 # when numbers cannot be stored exactly in bit system

Test Failed at In[5]:1
  Expression: 0.1 + 0.2 == 0.3
   Evaluated: 0.30000000000000004 == 0.3



LoadError: [91mThere was an error during testing[39m

In [6]:
# For floating-point approximations, you can use the ≈ operator, type it with \approx TAB
@test 0.1 + 0.2 ≈ 0.3 

Test Passed

In [7]:
@test π ≈ 3.14 atol=0.01 # if you play around and decrease atol, you will get a fail eventually

Test Passed

In [8]:
@test isapprox(0.1 + 0.2, 0.3)

Test Passed

###  Test Trows:
#### Test throws can be used to check for certain types of errors

In [9]:
 @test_throws DomainError sqrt(-1)

Test Passed
      Thrown: DomainError

In [10]:
sqrt(-1)

LoadError: DomainError with -1.0:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [11]:
@test_throws DimensionMismatch [1, 2, 3] + [1, 2]

Test Passed
      Thrown: DimensionMismatch

## The @testset macro 
This is a nice way to group together tests that are related, and it provides a nice structured output. @testset creates a local scope when running tests. All tests will be run even if an intermediate test fails. This is helpful in providing an overview of all the bugs in your code, isntead of stopping after the first failure.

You can explore changing the output and behavior of testset using test types (verbose, showtiming, failfast).

In [26]:
@testset "Arithmetic Tests" begin
    @test 1 + 1 == 2
    @test 2 * 3 == 6
    @test 4 - 2 == 2
end;

Test Summary:    | Pass  Total  Time
Arithmetic Tests |    3      3  0.0s


In [27]:
@testset "Arithmetic Tests" begin
    @test 1 + 1 == 20000 # even though this fails, testing continues 
    @test 2 * 3 == 6 
    @test 4 - 2 == 2
end;

Arithmetic Tests: Test Failed at In[27]:2
  Expression: 1 + 1 == 20000
   Evaluated: 2 == 20000

Stacktrace:
 [1] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:679 [inlined]
 [2] macro expansion
   @ In[27]:2 [inlined]
 [3] macro expansion
   @ ~/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1704 [inlined]
 [4] top-level scope
   @ In[27]:2
Test Summary:    | Pass  Fail  Total  Time
Arithmetic Tests |    2     1      3  0.0s


LoadError: [91mSome tests did not pass: 2 passed, 1 failed, 0 errored, 0 broken.[39m

### For loop testing

In [ ]:
@testset for i in 1:3
    @test i > 0
end;

### Nesting tests

In [28]:
foo(x) = length(x)^2

@testset verbose = true "Foo Tests" begin
           @testset "Animals" begin
               @test foo("cat") == 9
               @test foo("dog") == foo("cat")
           end
           @testset "Arrays $i" for i in 1:3
               @test foo(zeros(i)) == i^2
               @test foo(fill(1.0, i)) == i^2
           end
       end;

Test Summary: | Pass  Total  Time
Foo Tests     |    8      8  0.0s
  Animals     |    2      2  0.0s
  Arrays 1    |    2      2  0.0s
  Arrays 2    |    2      2  0.0s
  Arrays 3    |    2      2  0.0s


In [31]:
length("cat")^2

9

# Julia Package Testing Demo

I create a demo pacakge called Example (available on my GitHub below). Feel free to clone the repo and follow along! I will be showing you how to setup test files within a pacakge and use GitHub Actions to automatically run you tests each time you push your code :) 


## 1. Forking & Cloning the Repository

### Instructions:
1. **Go to my Example GitHub repository** (e.g., `https://github.com/kirstenlandsiedel/Example`).
2. **Clone** onto your local machine:
   ```sh
   git clone https://github.com/YOUR_GITHUB_USERNAME/Example.git
   cd Example
   ```

---

## 2. Understanding the Package Structure

### Key Files:
- **`src/Example.jl`** → Defines the module and functions.
- **`test/runtests.jl`** → Runs all test files.
- **`test/math_tests.jl` & `test/greeting_tests.jl`** → Contain specific test cases.
- **`Project.toml`** → Contains package metadata.
- **`.github/workflows`** → Contains code for GitHub Actions (auto-testing).

---

## 3. Walking Through the Code

### (A) `Example.jl`: The Main Code
```julia
module Example

function greet()
    "Hello world!"
end

function simple_add(a, b)
    a + b
end

function type_multiply(a::Float64, b::Float64)
    a * b
end

export greet, simple_add, type_multiply

end
```
📌 **Key Concepts:**
- The package is wrapped in a `module Example` block.
- Functions are defined and exported using `export`.

---

### (B) The Test Files

#### `test/math_tests.jl`
```julia
@testset "Testset 1" begin
    @test 2 == simple_add(1, 1)
    @test 3.5 == simple_add(1, 2.5)
    @test_throws MethodError simple_add(1, "A")
    @test_throws MethodError simple_add(1, 2, 3)
end

@testset "Testset 2" begin
    @test 1.0 == type_multiply(1.0, 1.0)
    @test isa(type_multiply(2.0, 2.0), Float64)
    @test_throws MethodError type_multiply(1, 2.5)
end
```
📌 **Key Concepts:**
- `@test` checks if a function produces the expected result.
- `@test_throws` ensures invalid inputs raise errors.

#### `test/greeting_tests.jl`
```julia
@testset "Testset 3" begin
    @test "Hello world!" == greet()
    @test_throws MethodError greet("Antonia")
end
```
📌 **Key Concepts:**
- The greeting function should return `"Hello world!"`
- Calling it with an argument should cause an error.

---

## 4. Running the Functions

### In terminal: julia()

```julia
using Pkg
Pkg.activate(".") # inside the Example folder 
using Example

greet()
simple_add(3, 5)
type_multiply(2.0, 4.0)
```

### In Jupyter Notebook

```julia
using Pkg
Pkg.activate("/Users/kirstenlandsiedel/Example")

using Example
greet()
simple_add(3, 5)
type_multiply(2.0, 4.0)
```


📌 **Think about::**  
- What happens if you run `simple_add(1, "hello")`?
- Why does `type_multiply(1, 2.5)` fail?

---

## 5. Running the Tests
Check if the package is working correctly.

```julia
Pkg.test()
```
📌 **Key Concept:**
- Julia finds and runs all test files inside `test/`.
- The output will show **which tests passed or failed**.

---

## **GitHub Actions for Automatic Testing**
If you have time, you can show them how to automatically run tests on **GitHub Actions** when they push code.

Would you like me to add a GitHub Actions setup for automated testing? 🚀


### **Setting Up GitHub Actions**

To automatically run tests on every push, create a GitHub Actions workflow:

1. **Create the GitHub Actions Workflow File**
   ```sh
   mkdir -p .github/workflows
   nano .github/workflows/ci.yml
   ```

2. **Add the following content:**

   ```yaml
   name: Run Julia Tests

   on:
     push:
       branches:
         - main
     pull_request:

   jobs:
     test:
       runs-on: ubuntu-latest

       steps:
         - name: Check out repository
           uses: actions/checkout@v4

         - name: Set up Julia
           uses: julia-actions/setup-julia@v2
           with:
             version: '1.11'

         - name: Verify Project.toml Exists
           run: |
             ls -l $GITHUB_WORKSPACE  # Debugging step to verify Project.toml is present
             cat $GITHUB_WORKSPACE/Project.toml || echo "Project.toml not found"

         - name: Install dependencies
           run: |
             cd $GITHUB_WORKSPACE
             julia -e 'using Pkg; Pkg.activate("."); Pkg.instantiate()'

         - name: Run tests
           run: |
             cd $GITHUB_WORKSPACE
             julia -e 'using Pkg; Pkg.activate("."); Pkg.test()'
   ```

3. **Save the file and commit it:**
   ```sh
   git add .github/workflows/ci.yml
   git commit -m "Added GitHub Actions workflow for testing"
   git push origin main
   ```
   

## 6. Challenge: Writing Their Own Tests
**Exercise:** Let's try to add a function we know will casue problems and see GitHub Actions in action catching what's happening. Note, I will edit these files from terminal. There are instructions on the commands I used to do so in section "Editing a File" below.

### **Add a new function to `src/Example.jl`**:

Remember to add buggy_divide to export statement

```julia
function buggy_divide(a, b)
    return a / b  # This function will fail if b == 0 (division by zero)
end
```

### **Create a test case in `test/math_tests.jl`**:
```julia
@testset "Buggy Divide Function" begin
    @test buggy_divide(10, 2) == 5  # This should pass
    @test_throws DivideError buggy_divide(10, 0)  # This will fail if not handled properly
end
```

### **Run the tests again**:
```julia
Pkg.test()
```
📌 **Ask them:**  
- What happens if they pass a string to `squared()`?
- Should they add `@test_throws` to catch that?

---

##  Editing a File 
**Goal:** Show students how to modify files using the terminal.

### Via Terminal: **Using `nano` (Command Line Editor)**
To edit a file, such as `src/Example.jl`, run:
```sh
nano src/Example.jl
```
- Use arrow keys to navigate.
- Make changes as needed.
- Save the file by pressing `CTRL + X`, then `Y`, then `Enter`.

### Via Github repo:
Probably a bad idea in general, but easier for playing around with Example if you want. 
- Click on file and then the pen icon. Edit away!


---


# Playing around with Example package

In [4]:
using Pkg
Pkg.activate("/Users/kirstenlandsiedel/Example")

  Activating project at `~/Example`


### Check out functions 

In [5]:
using Example
greet()

"Hello world!"

In [6]:
simple_add(3, 5)

8

In [7]:
type_multiply(2.0, 4.0)

8.0

### Run package test file

Take a look at the output! We can see that all tests are currently passing - wohoo!

In [9]:
Pkg.test()

     Testing Example
      Status `/private/var/folders/fr/mq93zz092j9b8xhtv0p9hc000000gn/T/jl_vF5Cqz/Project.toml`
  [08af438f] Example v0.1.0 `~/Example`
  [8dfed614] Test v1.11.0
      Status `/private/var/folders/fr/mq93zz092j9b8xhtv0p9hc000000gn/T/jl_vF5Cqz/Manifest.toml`
  [08af438f] Example v0.1.0 `~/Example`
  [2a0f44e3] Base64 v1.11.0
  [b77e0a4c] InteractiveUtils v1.11.0
  [56ddb016] Logging v1.11.0
  [d6f4376e] Markdown v1.11.0
  [9a3f8284] Random v1.11.0
  [ea8e919c] SHA v0.7.0
  [9e88b42a] Serialization v1.11.0
  [8dfed614] Test v1.11.0
     Testing Running tests...
Test Summary: | Pass  Total  Time
Example tests |    9      9  0.1s
     Testing Example tests passed 
